In [1]:
import os
import json
import csv
from datetime import datetime

In [2]:
curr_py_path = os.getcwd()
path = '\\DI_CONNECT\\'
path_addon = 'DI-Connect-User\\'
#path_addon = 'DI-Connect-Wellness'
#file_name = '2017-11-25_2018-03-05_61902345_sleepData.json'

In [ ]:
path = curr_py_path + path + path_addon
raw_string = r'{}'.format(path)
# Get all file names
files = os.listdir(raw_string)
files

In [4]:
file_paths = []
for file in files:
  # make file name 
  file_name = path + file
  file_paths.append(file_name)

# file_paths

In [5]:
# Get data
data = []

# loop through all files
for path in file_paths:
  # make path to raw string
  raw_string = r'{}'.format(path)

  # read file to csv
  with open(raw_string, 'r', encoding="utf8") as f_handle:
    json_dump = json.load(f_handle)
    
    # loop through all entries in file
    for entry in json_dump:
      try:
        # DATE AND TIME
        get_date = entry['calendarDate']["date"]
        date = datetime.strptime(get_date, '%b %d, %Y %I:%M:%S %p')
        date = date.strftime('%d-%b-%Y')
        
        # STEPS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        steps = entry["totalSteps"]
        step_goal = entry["dailyStepGoal"]
        distance_meters = entry["totalDistanceMeters"]

        # ACTIVE TIME ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        highly_active_secs = entry["highlyActiveSeconds"]
        active_secs = entry["activeSeconds"]
        
        try:
          vigorous_intensity_secs = entry["vigorousIntensityMinutes"] * 60
        except:
          vigorous_intensity_secs = None

        # CALORIES ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        total_cal = int(entry["totalKilocalories"])
        active_cal = int(entry["activeKilocalories"])
        bmr_cal = int(entry["bmrKilocalories"])

        # HEART RATE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        try:
          min_hr = entry["minHeartRate"]
          max_hr = entry["maxHeartRate"]
        except:
          min_hr, max_hr = None, None
          
        try:
          resting_hr = entry["restingHeartRate"]
        except:
          resting_hr = None
        
        try:
          min_avg_hr = entry["minAvgHeartRate"]
          max_avg_hr = entry["maxAvgHeartRate"]
        except:
          min_avg_hr, max_avg_hr = None, None
        
        # STRESS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        try:
          avg_stress_level = entry["allDayStress"]["aggregatorList"][0]["averageStressLevel"]
          avg_stress_level_intensity = entry["allDayStress"]["aggregatorList"][0]["averageStressLevelIntensity"]
          max_stress_lvl = entry["allDayStress"]["aggregatorList"][0]["maxStressLevel"]
        except:
          avg_stress_level, avg_stress_level_intensity, max_stress_lvl = None, None, None
        
        try:
          low_stress_secs = entry["allDayStress"]["aggregatorList"][0]["lowDuration"]
          medium_stress_secs = entry["allDayStress"]["aggregatorList"][0]["mediumDuration"]
          high_stress_secs = entry["allDayStress"]["aggregatorList"][0]["highDuration"]
        except:
          low_stress_secs, meduim_stress_secs, high_stress_secs = None, None, None
        
        # fixing unexplained errors
        if low_stress_secs is None:
          medium_stress_secs, high_stress_secs = None, None

        try:
          sum_LowMidHigh_stress_secs = entry["allDayStress"]["aggregatorList"][0]["stressDuration"]
          rest_secs = entry["allDayStress"]["aggregatorList"][0]["restDuration"]
          activity_secs = entry["allDayStress"]["aggregatorList"][0]["activityDuration"]
          uncategorised_stress_secs = entry["allDayStress"]["aggregatorList"][0]["uncategorizedDuration"]
          total_recorded_duration_secs = entry["allDayStress"]["aggregatorList"][0]["totalDuration"]
        except:
          sum_LowMidHigh_stress_secs, rest_secs, activity_secs, uncategorised_stress_secs, total_recorded_duration_secs = None, None, None, None, None


        to_save = {'date': date,
                   'steps': steps,
                   'step_goal': step_goal,
                   'distance_meters': distance_meters,
                   'highly_active_secs': highly_active_secs,
                   'active_secs': active_secs,
                   'vigorous_intensity_secs': vigorous_intensity_secs,
                   'total_cal': total_cal,
                   'active_cal': active_cal,
                   'bmr_cal': bmr_cal,
                   'min_hr': min_hr,
                   'max_hr': max_hr,
                   'resting_hr': resting_hr,
                   'min_avg_hr': min_avg_hr,
                   'max_avg_hr': max_avg_hr,
                   'avg_stress_level': avg_stress_level,
                   'avg_stress_level_intensity': avg_stress_level_intensity,
                   'max_stress_lvl': max_stress_lvl,
                   'low_stress_secs': low_stress_secs,
                   'medium_stress_secs': medium_stress_secs,
                   'high_stress_secs': high_stress_secs,
                   'sum_LowMidHigh_stress_secs': sum_LowMidHigh_stress_secs,
                   'rest_secs': rest_secs,
                   'activity_secs': activity_secs,
                   'uncategorised_stress_secs': uncategorised_stress_secs,
                   'total_recorded_duration_secs': total_recorded_duration_secs,
                  }

        data.append(to_save)
      except:
        pass

In [6]:
# Store data to CSV
# field names in new csv
field_names = ['date',
               'steps', 'step_goal', 'distance_meters',
               'active_secs', 'highly_active_secs', 'vigorous_intensity_secs',
               'total_cal', 'active_cal','bmr_cal',
               'min_hr', 'max_hr', 'resting_hr', 'min_avg_hr', 'max_avg_hr',
               'avg_stress_level', 'avg_stress_level_intensity', 'max_stress_lvl',
               'low_stress_secs', 'medium_stress_secs', 'high_stress_secs',
               'sum_LowMidHigh_stress_secs', 'rest_secs', 'activity_secs', 'uncategorised_stress_secs', 'total_recorded_duration_secs'
              ]

# Save data to new file
with open('daily_data.csv', 'w', encoding="utf8", newline='') as csv_file:
  writer = csv.DictWriter(csv_file, fieldnames=field_names)
  writer.writeheader()

  # loop through list of data
  for entry in data:
    writer.writerow({'date': entry['date'],
                   'steps': entry['steps'],
                   'step_goal': entry['step_goal'],
                   'distance_meters': entry['distance_meters'],
                   'highly_active_secs': entry['highly_active_secs'],
                   'active_secs': entry['active_secs'],
                   'vigorous_intensity_secs': entry['vigorous_intensity_secs'],
                   'total_cal': entry['total_cal'],
                   'active_cal': entry['active_cal'],
                   'bmr_cal': entry['bmr_cal'],
                   'min_hr': entry['min_hr'],
                   'max_hr': entry['max_hr'],
                   'resting_hr': entry['resting_hr'],
                   'min_avg_hr': entry['min_avg_hr'],
                   'max_avg_hr': entry['max_avg_hr'],
                   'avg_stress_level': entry['avg_stress_level'],
                   'avg_stress_level_intensity': entry['avg_stress_level_intensity'],
                   'max_stress_lvl': entry['max_stress_lvl'],
                   'low_stress_secs': entry['low_stress_secs'],
                   'medium_stress_secs': entry['medium_stress_secs'],
                   'high_stress_secs': entry['high_stress_secs'],
                   'sum_LowMidHigh_stress_secs': entry['sum_LowMidHigh_stress_secs'],
                   'rest_secs': entry['rest_secs'],
                   'activity_secs': entry['activity_secs'],
                   'uncategorised_stress_secs': entry['uncategorised_stress_secs'],
                   'total_recorded_duration_secs': entry['total_recorded_duration_secs']
                  })